In [ ]:
import os
import time
import pandas as pd
from dotenv import load_dotenv
import torch
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from bs4 import BeautifulSoup

In [ ]:
def start_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    service = Service('chromedriver.exe')  # Assumes chromedriver.exe is in the same folder
    driver = webdriver.Chrome(service=service, options=options)
    
    return driver

driver = start_driver()

In [ ]:
tqdm.pandas()

# Load model once
print("Loading AI model...")
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [ ]:
# Define your candidate labels
candidate_labels = ["food", "other"]


# Create a function to apply
def classify_message(text):
    if not isinstance(text, str) or text.strip() == "":
        return "unknown"  # or you can return None
    result = classifier(text, candidate_labels)
    return result["labels"][0]

In [ ]:
# Path to your JSON file
json_file_path = 'user_data_json\\tiktok\\user_data_tiktok.json'

# Step 1: Read the JSON
with open(json_file_path, 'r', encoding='ISO-8859-1') as file:
    data = json.load(file)

# Step 2: Print the data to understand its structure
print("Loaded JSON data:")


def fix_text(text):
    if isinstance(text, str):
        return text.encode('latin1').decode('utf-8')
    return text  # if it's not a string, just return it as is

def explore_json(data, indent=0):
    prefix = '  ' * indent
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{prefix}{key}: {type(value).__name__}")
            explore_json(value, indent + 1)
    elif isinstance(data, list) and data:
        print(f"{prefix}[List of {len(data)} items, type {type(data[0]).__name__}]")
        explore_json(data[0], indent + 1)

explore_json(data)

In [ ]:
messages = data["Direct Message"]["Direct Messages"]["ChatHistory"]["Chat History with quimerai:"]
df_zabella = pd.DataFrame(messages).sort_values('Date', ascending = False)
df_zabella['Content'] = df_zabella['Content'].apply(fix_text)
video_links = df_zabella.loc[df_zabella['Content'].str.contains('video')]
lst_video_links = list(dict.fromkeys(video_links['Content'].to_list()))

In [ ]:
data_description = []

print(f"We'll see {len(lst_video_links)} video descriptions")
for idx, video_url in enumerate(lst_video_links[:100]):
    driver.get(video_url)
    time.sleep(4)  # Wait for video page to load

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    desc_tag = soup.find('meta', attrs={'name': 'description'})

    if desc_tag:
        description = desc_tag['content']
        hashtags = [word for word in description.split() if word.startswith('#')]
        usernames = [word for word in description.split() if word.startswith('(@')]

        data_description.append({
            'video_url' : video_url,
            'description' : description,
            'hashtags' : hashtags,
            'usernames' : usernames,
        })

        print(f"{idx+1} out of {len(lst_video_links)} collected...")
        # print(f"🏷️ Hashtags: {hashtags}")
        # print(f"👥 Usernames: {usernames}")
        print("-" * 40)
    else:
        print('It couldnt collect anything')

df_scraped_tiktok = pd.DataFrame(data_description)
df_scraped_tiktok['food_related'] = df_scraped_tiktok['description'].progress_apply(classify_message)
df_scraped_tiktok.loc[df_scraped_tiktok['food_related'] == 'food'].to_csv('filtered_data.xlsx', index= False)

In [ ]:
df_filtered = pd.read_csv('filtered_data.csv')

In [ ]:
import yt_dlp
from pydub import AudioSegment
import librosa
import numpy as np

In [ ]:
transcribed_videos = []
def download_audio_from_tiktok(url, output_dir='downloadaudio'):
    os.makedirs(output_dir, exist_ok=True)

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_dir}/%(id)s.%(ext)s',
        'ffmpeg_location': 'C:/ffmpeg-2025-04-23-git-25b0a8e295-essentials_build/bin',
        'cookiefile': 'tiktok_cookies.txt',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        audio_path = os.path.join(output_dir, f"{info['id']}.mp3")
        return audio_path

def transcribe_with_huggingface(audio_path, model_name="openai/whisper-large-v3"):
    device = 0 if torch.cuda.is_available() else -1
    pipe = pipeline("automatic-speech-recognition", model=model_name, device=device)
    audio_full = AudioSegment.from_mp3(audio_path)
    below30sec = audio_full[:29000]
    samples = np.array(below30sec.get_array_of_samples()).astype(np.float32)
    # Normalize if stereo
    if below30sec.channels == 2:
        samples = samples.reshape((-1, 2))
        samples = samples.mean(axis=1)  # Convert to mono by averaging channels
    # Resample to 16kHz using librosa
    audio_input = librosa.resample(y=samples, orig_sr=below30sec.frame_rate, target_sr=16000)
    result = pipe(audio_input)
    return result["text"]

def process_tiktok_video(url):
    print(f"Processing: {url}")
    audio_path = download_audio_from_tiktok(url)
    transcript = transcribe_with_huggingface(audio_path)
    transcribed_videos.append({
        'video_url': url,
        'transcript': transcript
    })
    print(f"Transcript:\n{transcript}")
    return transcript

# Example usage


for url in df_filtered[:3]['video_url']:
    process_tiktok_video(url)

df_final = pd.DataFrame(transcribed_videos)

In [ ]:
df_complete = pd.merge(df_filtered, df_final, left_on=['video_url'], right_on=['url'], how = 'left')


In [ ]:
df_complete = df_complete.drop('url', axis=1)
df_complete.fillna(0, inplace=True)

In [ ]:
ner_pipeline = pipeline(
    "ner", 
    grouped_entities=True
)


In [ ]:
df_teste = df_complete.loc[df_complete['transcript'] != 0].copy()

In [ ]:
df_teste['entities'] = df_teste['transcript'].apply(lambda x: ner_pipeline(x))
df_teste['entities_description'] = df_teste['description'].apply(lambda x:ner_pipeline(x))

In [ ]:
df_teste

In [ ]:
def extract_places(entities):
    return [e['word'] for e in entities if e['entity_group'] in ['Local', 'ORG']]

df_teste['possible_places'] = df_teste['entities'].apply(extract_places)

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Descrições dos restaurantes
descriptions = [
    "Um rolê pra sair do óbvio em São Paulo. Vocês conhecem Nomihodai? É uma expressão japonesa que significa tudo que você puder beber. Basicamente é um happy hour japonês, como em vários bares e karaokês do Japão. Agora a gente tem a versão BR. Eu já falei do Koi aqui pra vocês, que fica na Santa Cecília. É bizarro. Pra quem não conhece, é um bar que também tem reserva no formato Makase no andar de cima. E agora eles tem o Nomihodai de terça a sexta, das 6h às 7h30. E como funciona? Tem soju, sake, cerveja e 4 drinks. Você pode beber à vontade por 120 reais. Além das bebidas, que eu já falo melhor delas.",
    "Um dos melhores brunchs de São Paulo fica escondido em um bairro residencial e só quem conhece sabe o quanto é bom. Tudo aqui é feito pela casa e um brunch completo sai por menos de R$ 65,00 por pessoa. E vem uma cesta de pães artesanais com acompanhamentos, ovos mexidos deliciosos com fatias crocantes de bacon, bolo caseiro do dia, cappuccino italiano ou tradicional brasileiro e a salada de frutas fresquinha. Já no Alacarte pedimos um toast de avocado com ovos mexidos e presunto.",
    "A gente foi jantar numa cafeteria japonesa com bonsai no teto, gelato de wasabi, matcha e os pratos mais bem servidos que eu já comi em São Paulo. Essa é a 1908 que fica no paraíso, literalmente no bairro Pará. E aqui a gente pediu gyoza e um tempurá de milho pra entrada. Eu já adianto que é muita coisa se você vai pedir o prato principal, porque olha isso. Eles são muito bem servidos e eles estavam tão bons quanto bonitos. Um yakisoba misto e um yakimichi, esse arroz frito com frango agridoce muito bom. Eu não sei dizer qual que foi o meu favorito, mas vale muito a pena pedir também."
]

# Definir as categorias possíveis para os restaurantes
ner = pipeline("ner", grouped_entities=True)
ner(descriptions[2])

In [ ]:
df_m = pd.json_normalize(data['messages'])
df_m.fillna(0, inplace=True)
df_m['share.share_text'] = df_m['share.share_text'].apply(fix_text)
df_m['sender_name'] = df_m['sender_name'].apply(fix_text)
df_m = df_m.loc[df_m['share.link'].str.contains('reel', na=False)].copy().reset_index()
df_m['food related'] = df_m['share.share_text'].progress_apply(classify_message)